In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Download reviews.txt and labels.txt from here: https://github.com/udacity/deep-learning/tree/master/sentiment-network

def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('gdrive/My Drive/Grokking/reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('gdrive/My Drive/Grokking/labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()


# Preprocess dataset:

import sys

f = open('gdrive/My Drive/Grokking/reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('gdrive/My Drive/Grokking/labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [34]:
print(len(raw_reviews),raw_reviews[:2])

25000 ['bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \n', 'story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra aud

In [0]:
print("tokens len", len(tokens),tokens[0])

tokens len 25000 {'', 'that', 'is', 's', 'to', 'teaching', 'sack', '\n', 'their', 'which', 'pity', 'classic', 'students', 'see', 'profession', 'such', 'when', 'teachers', 'expect', 'it', 'at', 'cartoon', 'a', 'schools', 'line', 'your', 'one', 'immediately', 'what', 'bromwell', 'life', 'my', 'of', 'survive', 'closer', 'who', 'many', 'tried', 'programs', 'can', 'age', 'lead', 'through', 'repeatedly', 'financially', 'all', 'pomp', 'situation', 'pettiness', 't', 'pathetic', 'same', 'high', 'other', 'than', 'isn', 'much', 'reality', 'adults', 'me', 'and', 'years', 'episode', 'inspector', 'far', 'whole', 'believe', 'right', 'scramble', 'about', 'fetched', 'i', 'here', 'in', 'm', 'burn', 'insightful', 'some', 'student', 'welcome', 'recalled', 'school', 'as', 'time', 'ran', '.', 'think', 'comedy', 'saw', 'the', 'satire', 'remind', 'down', 'knew'}


In [0]:
print("len(vocab)", len(vocab),"\n",vocab[:10])

len(vocab) 74074 
 ['reimagined', 'milner', 'conchatta', 'signboard', 'worn', 'divergence', 'rocket', 'disloyal', 'degenerative', 'amend']


In [0]:
print(len(input_dataset),input_dataset[0:2])

25000 [[49154, 52742, 70675, 15382, 41000, 53809, 53297, 4149, 71745, 28737, 1601, 73284, 2118, 11334, 58440, 64590, 33360, 18515, 30808, 47199, 26219, 73333, 54904, 13945, 21129, 26762, 3210, 19095, 47257, 25242, 66204, 56483, 6824, 22187, 12465, 28858, 7869, 59585, 9414, 16593, 46294, 62678, 72407, 67286, 5850, 24799, 63717, 15612, 20224, 53003, 15634, 56082, 20754, 25878, 58648, 42270, 19746, 16706, 51529, 40782, 63320, 57689, 46428, 25440, 57718, 12665, 11647, 61312, 54152, 19859, 63898, 66475, 22445, 51122, 21428, 71608, 33722, 62395, 17859, 1988, 72648, 32207, 24015, 41940, 8149, 45013, 60891, 57307, 44005, 2545, 22514, 60412, 5118], [51712, 49154, 54275, 2057, 26122, 4118, 16918, 6177, 59938, 34343, 41000, 55358, 7747, 11334, 64072, 58440, 70734, 18515, 6231, 57950, 35429, 12396, 23160, 30329, 67706, 21129, 3210, 62606, 16018, 25242, 65188, 36019, 64185, 7869, 41150, 57022, 59585, 73417, 68306, 9939, 30933, 46294, 5850, 43228, 16606, 35046, 37102, 1774, 12529, 51954, 39668, 3609

In [0]:
import sys,random,math
from collections import Counter
import numpy as np

hidden_size,window,negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2  #shape: (74075, 50)
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0              #shape: (74075, 50)

In [60]:
import numpy as np
norms = np.sum(weights_0_1 * weights_0_1,axis=1)                    #norms.shape (74074,)
norms.resize(norms.shape[0],1)                                      #norms.shape (74074,1)
normed_weights = weights_0_1 * norms                                #normed_weights.shape (74074, 50)

def make_sent_vect(words):
    indices = list(map(lambda x:word2index[x],filter(lambda x:x in word2index,words)))  #word2index len: 74074, eg -> word2index['cat']:63378
                                                                                        #indices[:5] -> [24845, 68066, 23325, 47974, 69814]
    return np.mean(normed_weights[indices],axis=0)                  #shape: (50,)

reviews2vectors = list()
for review in tokens: # tokenized reviews
    reviews2vectors.append(make_sent_vect(review))
reviews2vectors = np.array(reviews2vectors)                         #reviews2vectors.shape: (25000, 50)

def most_similar_reviews(review):
    v = make_sent_vect(review)                                      #v.shape: (50,)
    scores = Counter()
    for i,val in enumerate(reviews2vectors.dot(v)):                 #reviews2vectors.dot(v).shape: (25000,)
        scores[i] = val
    most_similar = list()
    
    for idx,score in scores.most_common(3):
        most_similar.append(raw_reviews[idx][0:40])
    return most_similar

most_similar_reviews(['boring','awful'])

['as i saw the movie i was really shocked ',
 'an annoying experience . improvised dial',
 'this is waaaaay to much . . so frustrati']